## Random Forest Model using Nevergrad gradient-free optimizer for hypertuning
This notebook aims to compare the performance of RF models using gridsearchcv hyperparameters tuning vs using Nevergrad. The experiment below is very "lite" and could only serve as a basis for further experimentation. This notebook is explained in a Toky Axel's Medium article : 

In [1]:
# Needed packages
#!pip install tqdm
#!pip install nevergrad

In [2]:
import tqdm
import pandas as pd
import nevergrad as ng
from scipy.io.arff import loadarff 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

### 1. Dataset sampling
More details about the dataset : https://www.openml.org/search?type=data&id=1494&sort=runs&status=active

In [3]:
raw = loadarff('qsar-biodeg.arff')
data = pd.DataFrame(raw[0])
data = data.replace({'Class': {b'2': 1, b'1': 0}})

In [4]:
data.shape

(1055, 42)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data.drop("Class",axis=1), data.Class, test_size = 0.33, random_state = 123)

### 2. Model training using gridsearchcv hypertuning
More details about the model : https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [6]:
y_train.shape

(706,)

In [7]:
y_train.value_counts()

Class
0    471
1    235
Name: count, dtype: int64

In [8]:
y_test.value_counts()

Class
0    228
1    121
Name: count, dtype: int64

In [9]:
tuned_parameters = {
    'n_estimators': [10, 100, 200], 
    'max_depth' : [None, 10, 20], 
    'min_samples_split' : range(2,50,20),
    'max_features' :['sqrt','log2', None]
}
clf = GridSearchCV(RandomForestClassifier(random_state=42), tuned_parameters, cv=5, scoring="roc_auc")
clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': range(2, 50, 20),
                         'n_estimators': [10, 100, 200]},
             scoring='roc_auc')

In [10]:
clf.best_params_

{'max_depth': 10,
 'max_features': 'log2',
 'min_samples_split': 2,
 'n_estimators': 200}

In [11]:
clf.best_score_ 

0.9184739462962522

In [17]:
best_clf = RandomForestClassifier(random_state=42,**clf.best_params_)
best_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=200,
                       random_state=42)

#### Evaluate the model on Test set & Train set :  
The train AUC score below can lead us to overfitting

In [18]:
roc_auc_score(y_test, best_clf.predict_proba(X_test)[:,1])

0.9464622299550529

In [19]:
roc_auc_score(y_train, best_clf.predict_proba(X_train)[:,1])

0.9999819307042508

### 3. Model training using nevergrad
More details about the method : https://facebookresearch.github.io/nevergrad/machinelearning.html

We build a custom score that prevents us from overfitting : 
$$score = 1-auc\_on\_test + (w*abs((1-auc\_on\_train)-(auc\_on\_test)))$$
where $$ w = fixed\_weight\ (example\ 3) $$

In [24]:
def train_and_return_test_error(params, X_train, y_train, X_test, y_test):
    rf_clf = RandomForestClassifier(random_state=42, **params)
    rf_clf.fit(X_train,y_train)
    score_on_test = 1-roc_auc_score(y_test, rf_clf.predict_proba(X_test)[:,1])
    score_on_train = 1-roc_auc_score(y_train, rf_clf.predict_proba(X_train)[:,1])
    return [score_on_test, score_on_train]

# Parametrization is how nevergrad configures the optimizers.
# (https://facebookresearch.github.io/nevergrad/parametrization.html)
parametrization = ng.p.Dict(
    # Discrete params bounded by lower and upper bounds
    n_estimators = ng.p.Scalar(init=10, lower=2, upper=1000).set_integer_casting(),
    max_depth = ng.p.Scalar(init=None, lower=2, upper=1000).set_integer_casting(),
    min_samples_split = ng.p.Scalar(init=2, lower=2, upper=1000).set_integer_casting(),
    # Choice between given values.
    max_features=ng.p.Choice(['sqrt','log2', None]),
)

budget = 1000  # How many trainings we will do before concluding.

# List of some available optimizers
names = ["RandomSearch", "TwoPointsDE", "CMA", "PSO", "ScrHammersleySearch"]

In [25]:
all_recommendation = {}

for name in names:
    optim = ng.optimizers.registry[name](parametrization=parametrization, 
                                         budget=budget, 
                                         num_workers=3
                                        )
    
    # Evaluate the model each 10 iterations
    for u in tqdm.tqdm(range(budget // 10)):
        # Ask and tell can be asynchronous.
        # Just be careful that you "tell" something that was asked.
        # Here we ask 3 times and tell 3 times in order to fake asynchronicity
        x1 = optim.ask()
        x2 = optim.ask()
        x3 = optim.ask()
        # The three following lines could be parallelized.
        # We could also do things asynchronously, i.e. do one more ask
        # as soon as a training is over.
        y1 = train_and_return_test_error(x1.value, X_train, y_train, X_test, y_test)
        y2 = train_and_return_test_error(x2.value, X_train, y_train, X_test, y_test)
        y3 = train_and_return_test_error(x3.value, X_train, y_train, X_test, y_test)
        # We want to minimize the test AUC 
        # but also the difference between train and test AUC
        # here we use Fixed Weight (3 times) for the difference
        y1_hat = y1[0]+(3*(abs(y1[1]-y1[0])))
        y2_hat = y2[0]+(3*(abs(y2[1]-y2[0])))
        y3_hat = y3[0]+(3*(abs(y3[1]-y3[0])))
        # Tell to optimizers the value of the loss function
        optim.tell(x1, y1_hat)
        optim.tell(x2, y2_hat)
        optim.tell(x3, y3_hat)
        
    recommendation = optim.recommend()
    score = train_and_return_test_error(recommendation.value, X_train, y_train, X_test, y_test)
    all_recommendation.update({name:recommendation.value})
    
    print(name, " provides a vector of parameters", 
          recommendation.value," with AUC on test :",
          1-score[0]," (AUC on train  ",1-score[1],")")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [07:45<00:00,  4.65s/it]


RandomSearch  provides a vector of parameters {'n_estimators': 7, 'max_depth': 332, 'min_samples_split': 80, 'max_features': 'log2'}  with AUC on test : 0.9209076410033348  (AUC on train   0.9267967655960608 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [07:47<00:00,  4.67s/it]


TwoPointsDE  provides a vector of parameters {'n_estimators': 210, 'max_depth': 304, 'min_samples_split': 150, 'max_features': 'log2'}  with AUC on test : 0.9027475714078584  (AUC on train   0.9058047612594299 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [12:39<00:00,  7.60s/it]


CMA  provides a vector of parameters {'n_estimators': 217, 'max_depth': 497, 'min_samples_split': 149, 'max_features': 'log2'}  with AUC on test : 0.9025300855444396  (AUC on train   0.9061661471744138 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [10:24<00:00,  6.25s/it]


PSO  provides a vector of parameters {'n_estimators': 5, 'max_depth': 685, 'min_samples_split': 76, 'max_features': 'log2'}  with AUC on test : 0.9156879802812817  (AUC on train   0.922667931517369 )


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [14:05<00:00,  8.46s/it]


ScrHammersleySearch  provides a vector of parameters {'n_estimators': 109, 'max_depth': 29, 'min_samples_split': 83, 'max_features': 'log2'}  with AUC on test : 0.9256198347107438  (AUC on train   0.9405746036048246 )


In [64]:
all_recommendation

{'RandomSearch': {'n_estimators': 17,
  'max_depth': 582,
  'min_samples_split': 106,
  'max_features': 'sqrt'},
 'OnePlusOne': {'n_estimators': 57,
  'max_depth': 193,
  'min_samples_split': 118,
  'max_features': 'sqrt'},
 'CMA': {'n_estimators': 65,
  'max_depth': 311,
  'min_samples_split': 104,
  'max_features': None},
 'PSO': {'n_estimators': 36,
  'max_depth': 702,
  'min_samples_split': 68,
  'max_features': 'sqrt'},
 'ScrHammersleySearch': {'n_estimators': 10,
  'max_depth': 78,
  'min_samples_split': 89,
  'max_features': 'log2'}}